Этап 1. Реализация базового класса Account \
Класс должен быть инициализирован с параметрами: \
1) account_holder (str): имя владельца счёта; \
2) balance (float, по умолчанию 0): начальный баланс счёта, не может быть
отрицательным. \
Атрибуты: \
1) holder: хранит имя владельца; \
2) _balance: приватный атрибут для хранения текущего баланса; \
3) operations_history: список или другая структура для хранения истории
операций. \
Важно: каждая операция должна храниться не просто как число, а как
структурированная информация, например, словарь или кортеж. Минимальный набор
данных для операции: тип операции ('deposit' или 'withdraw'), сумма, дата и время
операции, текущий баланс после операции, статус ('success' или 'fail').

In [16]:
from datetime import datetime
from enum import Enum

# Типы операций (для удобства и чтобы не было опечаток в строках)
class OperationType(str, Enum):
    DEPOSIT = "deposit"
    WITHDRAW = "withdraw"

# Статус выполнения операции
class Status(str, Enum):
    SUCCESS = "success"
    FAIL = "fail"

class Account:
    holder: str
    _balance: float
    operations_history: list[dict]
    
    def __init__(self, account_holder: str, balance: float = 0.0):
        # Инициализация счёта: владелец, текущий баланс, история операций
        self.holder = account_holder
        self._balance = balance
        self.operations_history = []
    
    def deposit(self, amount):
        if amount > 0:
            self._balance += amount
            # Фиксируем успешную операцию в истории
            self.operations_history.append(self.write_operation(
                operation_type=OperationType.DEPOSIT.value,
                amount=amount,
                status=Status.SUCCESS.value
            ))
        else: 
            raise ValueError(f"Wrong amount: {amount}")

    def withdraw(self, amount):
        if amount > 0:
            if self._balance >= amount:
                self._balance -= amount
                status = Status.SUCCESS.value
            else: 
                # Денег недостаточно — баланс не меняется, но попытка фиксируется
                status = Status.FAIL.value
                
            self.operations_history.append(self.write_operation(
                operation_type=OperationType.WITHDRAW.value,
                amount=amount,
                status=status,
            ))
        else:
            raise ValueError(f"Wrong amount: {amount}")
    
    def get_balance(self):
        return self._balance

    def get_history(self):
        return self.operations_history
    
    def write_operation(self, operation_type: OperationType, amount: float, status: Status,used_credit: bool = False):
        if operation_type not in [OperationType.DEPOSIT, OperationType.WITHDRAW]:
            raise ValueError(f"Unknown operation: {operation_type}")
        
        # Запись операции: тип, сумма, время, баланс после операции, статус + признак кредита (для CreditAccount)
        operation = {
            "operation_type": operation_type,
            "amount": amount,
            "date": datetime.now().isoformat(sep=" ", timespec="seconds"),
            "balance_after": self._balance,
            "used_credit": used_credit,
            "status": status
        }

        return operation

In [17]:
class CreditAccount(Account):
    credit_limit: float

    def __init__(self, account_holder, balance: float = 0.0, credit_limit: float = 0.0):
        super().__init__(account_holder, balance)
        self.credit_limit = credit_limit
    
    def get_avaliable_money(self):
        return self.get_balance() + self.credit_limit
    
    def withdraw(self, amount: float):
        # Снятие в кредитном счёте: можно уходить в минус, но не ниже -credit_limit
        if amount <= 0:
            raise ValueError(f"Wrong amount: {amount}")
        
        balance_before = self._balance
        if amount <= self.get_avaliable_money():
            self._balance -= amount
            status = Status.SUCCESS.value
            # Кредит использован, если суммы на своём балансе было недостаточно
            used_credit = amount > balance_before
        else:
            # Не хватает даже с учётом кредитного лимита
            status = Status.FAIL.value
            used_credit = False

        self.operations_history.append(self.write_operation(
            operation_type=OperationType.WITHDRAW.value,
            amount=amount,
            status=status,
            used_credit=used_credit
        ))

In [18]:
acc = CreditAccount(account_holder="Dmitriy", balance=100.0, credit_limit=200.0)

acc.withdraw(50.0)    # 1) без кредита
acc.withdraw(200.0)   # 2) с кредитом (баланс станет отрицательным)
acc.withdraw(100.0)   # 3) fail (даже с кредитом не хватает)

print(acc.get_balance())
for op in acc.get_history():
    print(op)


-150.0
{'operation_type': 'withdraw', 'amount': 50.0, 'date': '2025-12-21 03:34:56', 'balance_after': 50.0, 'used_credit': False, 'status': 'success'}
{'operation_type': 'withdraw', 'amount': 200.0, 'date': '2025-12-21 03:34:56', 'balance_after': -150.0, 'used_credit': True, 'status': 'success'}
{'operation_type': 'withdraw', 'amount': 100.0, 'date': '2025-12-21 03:34:56', 'balance_after': -150.0, 'used_credit': False, 'status': 'fail'}
